# Sichten und Zugriffskontrolle

In [ ]:
%CREATE pg://localhost OF tables.sql

In [ ]:
--%CREATE :memory: OF tables.sql
-- DuckDB unterstützt keine Updates auf Views!

In [ ]:
%SCHEMA

## 10-8

In [ ]:
SELECT * FROM Weine NATURAL JOIN Erzeuger

In [ ]:
CREATE VIEW Rotweine AS (
    SELECT WeinID, Name, Jahrgang, Weingut
    FROM Weine
    WHERE Farbe = 'Rot'
)

In [ ]:
SELECT * FROM Rotweine

Unterschied zur CTE:

In [ ]:
WITH Rotweine_temp AS (
    SELECT WeinID, Name, Jahrgang, Weingut
    FROM Weine
    WHERE Farbe = 'Rot'
)

SELECT * FROM Rotweine_temp

In [ ]:
SELECT * FROM Rotweine_temp

## 10-10

In [ ]:
UPDATE Rotweine SET Jahrgang = 2025

In [ ]:
SELECT * FROM Rotweine

In [ ]:
SELECT * FROM Weine

## 10-12

In [ ]:
INSERT INTO Rotweine VALUES (42000, 'Neuer Wein', 2025, 'Müller')

In [ ]:
SELECT * FROM Rotweine

In [ ]:
SELECT * FROM Weine

## 10-14ff

ohne `WITH CHECK OPTION`

In [ ]:
CREATE VIEW neue_weine AS (
    SELECT *
    FROM Weine
    WHERE Jahrgang = 2025
)

In [ ]:
SELECT * FROM neue_weine

In [ ]:
UPDATE neue_weine SET Jahrgang = 2000 WHERE Name = 'Neuer Wein'

In [ ]:
SELECT * FROM neue_weine

mit `WITH CHECK OPTION`

In [ ]:
CREATE OR REPLACE VIEW neue_weine AS (
    SELECT *
    FROM Weine
    WHERE Jahrgang = 2025
)
WITH CHECK OPTION

In [ ]:
SELECT * FROM neue_weine

In [ ]:
UPDATE neue_weine SET Jahrgang = 2000 WHERE Name = 'Zinfandel'

## 10-24

In [ ]:
CREATE VIEW v_weinerzeuger AS (
    SELECT *
    FROM Weine
    NATURAL JOIN Erzeuger
)

In [ ]:
SELECT * FROM v_weinerzeuger

In [ ]:
CREATE OR REPLACE FUNCTION weinerzeuger_insert_function()
    RETURNS TRIGGER AS
$func$
BEGIN
    INSERT INTO WEINE
    VALUES (NEW.WeinID, NEW.Name, NEW.Farbe, NEW.Jahrgang, NEW.Weingut);
    RETURN NULL;
END
$func$ LANGUAGE plpgsql;

CREATE OR REPLACE TRIGGER V_WEINERZEUGER_Insert
INSTEAD OF INSERT ON V_WEINERZEUGER
FOR EACH ROW
EXECUTE FUNCTION weinerzeuger_insert_function();

In [ ]:
-- Einfügen mit bekanntem Erzeuger
INSERT INTO v_weinerzeuger
VALUES ('Helena', 3333, 'Dornfelder', 'Rot', 2002, 'Barossa Valley', 'Südaustralien')

In [ ]:
SELECT * FROM v_weinerzeuger

In [ ]:
-- Einfügen mit unbekanntem Erzeuger
INSERT INTO v_weinerzeuger
VALUES ('Unbekannter Erzeuger', 3333, 'Dornfelder', 'Rot', 2002, 'Barossa Valley', 'Südaustralien')

## 10-25

In [ ]:
CREATE OR REPLACE VIEW aggr1 AS (
    SELECT Farbe, AVG(Jahrgang) AS Jahrgang
    FROM Weine
    WHERE Farbe IS NOT NULL
    GROUP BY Farbe
    HAVING AVG(Jahrgang) > 2010
)

In [ ]:
SELECT Farbe FROM aggr1 WHERE Jahrgang > 2015

In [ ]:
CREATE OR REPLACE VIEW aggr2 AS (
    SELECT Farbe, AVG(Jahrgang) AS Jahrgang
    FROM Weine
    WHERE Farbe IS NOT NULL
    GROUP BY Farbe
)

In [ ]:
SELECT MIN(Jahrgang) FROM aggr2